# Seattle Weather Prediction Model

This notebook demonstrates how to build a machine learning model to predict weather conditions based on meteorological features.

## Dataset Features:
- **date**: Date of observation
- **precipitation**: Amount of precipitation (mm)
- **temp_max**: Maximum temperature (°C)
- **temp_min**: Minimum temperature (°C)
- **wind**: Wind speed (m/s)
- **weather**: Weather type (drizzle, rain, sun, snow, fog) - **TARGET VARIABLE**

## Step 1: Upload the CSV File to Google Colab

Run the cell below to upload your `seattle-weather.csv` file.

In [ ]:
# Upload the CSV file
from google.colab import files
uploaded = files.upload()
# After running this cell, click 'Choose Files' and select 'seattle-weather.csv'

## Step 2: Install and Import Required Libraries

In [ ]:
# Install required libraries (if not already installed)
!pip install pandas numpy scikit-learn matplotlib seaborn

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## Step 3: Load and Explore the Data

In [ ]:
# Load the dataset
df = pd.read_csv('seattle-weather.csv')

# Display basic information
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head(10))

print("\nDataset Info:")
print(df.info())

print("\nStatistical Summary:")
print(df.describe())

print("\nMissing Values:")
print(df.isnull().sum())

In [ ]:
# Explore the target variable (weather)
print("Weather Types Distribution:")
print(df['weather'].value_counts())

# Visualize weather distribution
plt.figure(figsize=(10, 6))
df['weather'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribution of Weather Types', fontsize=16, fontweight='bold')
plt.xlabel('Weather Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Step 4: Data Visualization and Analysis

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of features by weather type
features = ['precipitation', 'temp_max', 'temp_min', 'wind']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, feature in enumerate(features):
    for weather_type in df['weather'].unique():
        data = df[df['weather'] == weather_type][feature]
        axes[idx].hist(data, alpha=0.5, label=weather_type, bins=30)
    
    axes[idx].set_xlabel(feature.replace('_', ' ').title(), fontsize=12)
    axes[idx].set_ylabel('Frequency', fontsize=12)
    axes[idx].set_title(f'Distribution of {feature.replace("_", " ").title()}', fontsize=14)
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Box plots for each feature by weather type
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, feature in enumerate(features):
    sns.boxplot(data=df, x='weather', y=feature, ax=axes[idx], palette='Set2')
    axes[idx].set_title(f'{feature.replace("_", " ").title()} by Weather Type', fontsize=14)
    axes[idx].set_xlabel('Weather Type', fontsize=12)
    axes[idx].set_ylabel(feature.replace('_', ' ').title(), fontsize=12)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Step 5: Data Preprocessing

In [ ]:
# Create a copy of the dataframe for processing
df_processed = df.copy()

# Convert date to datetime and extract useful features
df_processed['date'] = pd.to_datetime(df_processed['date'])
df_processed['year'] = df_processed['date'].dt.year
df_processed['month'] = df_processed['date'].dt.month
df_processed['day'] = df_processed['date'].dt.day
df_processed['day_of_week'] = df_processed['date'].dt.dayofweek
df_processed['day_of_year'] = df_processed['date'].dt.dayofyear

# Create additional features
df_processed['temp_range'] = df_processed['temp_max'] - df_processed['temp_min']
df_processed['is_freezing'] = (df_processed['temp_min'] <= 0).astype(int)

# Drop the original date column
df_processed = df_processed.drop('date', axis=1)

print("Processed dataset shape:", df_processed.shape)
print("\nNew features added!")
print(df_processed.head())

In [ ]:
# Separate features and target
X = df_processed.drop('weather', axis=1)
y = df_processed['weather']

print("Feature columns:")
print(X.columns.tolist())
print("\nFeatures shape:", X.shape)
print("Target shape:", y.shape)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("\nWeather type encoding:")
for i, weather_type in enumerate(label_encoder.classes_):
    print(f"{weather_type}: {i}")

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")
print(f"\nTraining set distribution:")
print(pd.Series(y_train).value_counts().sort_index())
print(f"\nTesting set distribution:")
print(pd.Series(y_test).value_counts().sort_index())

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled successfully!")
print(f"\nScaled training data shape: {X_train_scaled.shape}")
print(f"Scaled testing data shape: {X_test_scaled.shape}")

## Step 6: Train Multiple Models

In [ ]:
# Define multiple models to compare
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Support Vector Machine': SVC(kernel='rbf', random_state=42)
}

print("Models defined:")
for model_name in models.keys():
    print(f"  - {model_name}")

In [ ]:
# Train and evaluate all models
results = {}

print("Training models...\n")
print("="*70)

for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    
    # Train the model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy
    
    print(f"{model_name} Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print("-"*70)

print("\n" + "="*70)
print("All models trained successfully!")

## Step 7: Model Comparison and Evaluation

In [ ]:
# Visualize model comparison
plt.figure(figsize=(12, 6))
model_names = list(results.keys())
accuracies = list(results.values())

colors = plt.cm.viridis(np.linspace(0, 1, len(model_names)))
bars = plt.bar(model_names, accuracies, color=colors, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height*100:.2f}%',
             ha='center', va='bottom', fontweight='bold')

plt.title('Model Accuracy Comparison', fontsize=16, fontweight='bold')
plt.xlabel('Model', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1.1)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Find the best model
best_model_name = max(results, key=results.get)
best_accuracy = results[best_model_name]

print(f"\n🏆 Best Model: {best_model_name}")
print(f"🎯 Best Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")

In [ ]:
# Detailed evaluation of the best model
best_model = models[best_model_name]
y_pred_best = best_model.predict(X_test_scaled)

print(f"\n{'='*70}")
print(f"Detailed Evaluation: {best_model_name}")
print(f"{'='*70}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred_best, 
                          target_names=label_encoder.classes_))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            cbar_kws={'label': 'Count'})
plt.title(f'Confusion Matrix - {best_model_name}', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Weather', fontsize=12)
plt.ylabel('Actual Weather', fontsize=12)
plt.tight_layout()
plt.show()

## Step 8: Feature Importance Analysis

In [ ]:
# Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance, x='importance', y='feature', palette='viridis')
    plt.title(f'Feature Importance - {best_model_name}', fontsize=16, fontweight='bold')
    plt.xlabel('Importance Score', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print("\nTop 5 Most Important Features:")
    print(feature_importance.head())
else:
    print(f"Feature importance not available for {best_model_name}")

## Step 9: Make Predictions on New Data

In [ ]:
# Example: Make a prediction for custom weather data
def predict_weather(precipitation, temp_max, temp_min, wind, month, day):
    """
    Predict weather based on input parameters
    
    Parameters:
    - precipitation: Amount of precipitation (mm)
    - temp_max: Maximum temperature (°C)
    - temp_min: Minimum temperature (°C)
    - wind: Wind speed (m/s)
    - month: Month (1-12)
    - day: Day of month (1-31)
    """
    import datetime
    
    # Create a sample date for the given month and day
    year = 2023  # arbitrary year
    date = datetime.datetime(year, month, day)
    
    # Calculate derived features
    day_of_week = date.weekday()
    day_of_year = date.timetuple().tm_yday
    temp_range = temp_max - temp_min
    is_freezing = 1 if temp_min <= 0 else 0
    
    # Create feature array in the same order as training data
    features = np.array([[
        precipitation, temp_max, temp_min, wind,
        year, month, day, day_of_week, day_of_year,
        temp_range, is_freezing
    ]])
    
    # Scale features
    features_scaled = scaler.transform(features)
    
    # Make prediction
    prediction = best_model.predict(features_scaled)
    predicted_weather = label_encoder.inverse_transform(prediction)[0]
    
    # Get prediction probabilities if available
    if hasattr(best_model, 'predict_proba'):
        probabilities = best_model.predict_proba(features_scaled)[0]
        prob_dict = {label_encoder.classes_[i]: prob 
                    for i, prob in enumerate(probabilities)}
        return predicted_weather, prob_dict
    else:
        return predicted_weather, None

# Test the prediction function
print("Example Predictions:\n")
print("="*70)

# Example 1: Rainy conditions
print("\n1. Rainy conditions:")
print("   Input: precipitation=15.0, temp_max=10.0, temp_min=5.0, wind=5.0, month=11, day=15")
weather, probs = predict_weather(15.0, 10.0, 5.0, 5.0, 11, 15)
print(f"   Predicted Weather: {weather}")
if probs:
    print("   Probabilities:")
    for w, p in sorted(probs.items(), key=lambda x: x[1], reverse=True):
        print(f"     {w}: {p*100:.2f}%")

# Example 2: Sunny conditions
print("\n2. Sunny conditions:")
print("   Input: precipitation=0.0, temp_max=25.0, temp_min=15.0, wind=2.0, month=7, day=20")
weather, probs = predict_weather(0.0, 25.0, 15.0, 2.0, 7, 20)
print(f"   Predicted Weather: {weather}")
if probs:
    print("   Probabilities:")
    for w, p in sorted(probs.items(), key=lambda x: x[1], reverse=True):
        print(f"     {w}: {p*100:.2f}%")

# Example 3: Snowy conditions
print("\n3. Snowy conditions:")
print("   Input: precipitation=10.0, temp_max=0.0, temp_min=-5.0, wind=3.0, month=1, day=10")
weather, probs = predict_weather(10.0, 0.0, -5.0, 3.0, 1, 10)
print(f"   Predicted Weather: {weather}")
if probs:
    print("   Probabilities:")
    for w, p in sorted(probs.items(), key=lambda x: x[1], reverse=True):
        print(f"     {w}: {p*100:.2f}%")

print("\n" + "="*70)

## Step 10: Save the Trained Model (Optional)

In [ ]:
# Save the model and preprocessing objects
import pickle

# Save the best model
with open('weather_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save the label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("Model and preprocessing objects saved successfully!")
print(f"Best model: {best_model_name}")
print(f"Model file: weather_model.pkl")
print(f"Scaler file: scaler.pkl")
print(f"Label encoder file: label_encoder.pkl")

In [ ]:
# Download the saved models from Colab
from google.colab import files

files.download('weather_model.pkl')
files.download('scaler.pkl')
files.download('label_encoder.pkl')

print("Models downloaded! You can now use them for predictions in other applications.")

## Summary

### What We Accomplished:
1. ✅ Loaded and explored the Seattle weather dataset
2. ✅ Visualized data distributions and correlations
3. ✅ Engineered new features from the date column
4. ✅ Trained multiple machine learning models
5. ✅ Compared model performances
6. ✅ Evaluated the best model with detailed metrics
7. ✅ Analyzed feature importance
8. ✅ Created a prediction function for new data
9. ✅ Saved the trained model for future use

### Next Steps:
- Try different feature engineering techniques
- Experiment with hyperparameter tuning
- Use cross-validation for more robust evaluation
- Try deep learning models (Neural Networks)
- Deploy the model as a web application

### Key Insights:
- The model can predict weather types with good accuracy
- Temperature and precipitation are typically important features
- Different weather types have distinct patterns in the data

**Thank you for using this notebook! Happy predicting! 🌤️**